# HateXplain

https://huggingface.co/Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two


The model is used for classifying a text as Abusive (Hatespeech and Offensive) or Normal. The model is trained using data from Gab and Twitter and Human Rationales were included as part of the training data to boost the performance. The model also has a rationale predictor head that can predict the rationales given an abusive sentence.

The dataset and models are available here: https://github.com/punyajoy/HateXplain

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BatchEncoding
### from models.py
from models import *
import os
import shutil

import pandas as pd
import numpy as np

import tensorflow as tf

tf.get_logger().setLevel('ERROR')

seed = 42
sample_size = 30

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
model = Model_Rational_Label.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")

In [30]:
# reading in train and validaiton dataset
hate_data = pd.read_csv("datasets\hate speech tweets\labeled_data.csv").drop(columns=["Unnamed: 0", "count", "hate_speech", "offensive_language", "neither"]).rename(columns={'class' : 'label', "tweet":"text"})

# for human readability
class_names = ["hate speech", "offensive language", "neither"]

In [ ]:
# Source https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379

def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             padding='longest', #implements dynamic padding
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    

In [5]:
data = hate_data.text.sample(sample_size).values.tolist()


batchencoder = (tokenizer(data, return_tensors="pt", padding=True))

# print(batchencoder['input_ids'])
#                        padding='longest', #implements dynamic padding
#                        return_attention_mask=True,
#                        return_token_type_ids=False).convert_to_tensors("tf")

# input_ids = batchencoder.convert_to_tensors("tf")
# max_length = np.max(list(map(len, data)))
# print(max_length)
# input_ids, attention_mask = batch_encode(tokenizer=tokenizer, texts=data, max_length=max_length)
# print(input_ids)

In [6]:
prediction_logits, _ = model(input_ids=batchencoder['input_ids'],attention_mask=batchencoder['attention_mask'])

In [39]:
predicitons = torch.argmax(prediction_logits, dim=1)

In [40]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]} : label: {class_names[results[i]]}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()

print_my_examples(data, predicitons.tolist())

input: @BitchsLoveMosa do these bitches really love Mosa though #MosaGang http://t.co/0g1J8Bibpl : label: offensive language
input: That depression shit trash. : label: hate speech
input: RT @_____redd: killing these bitches go body for body &#128530;&#128524;&#127911; : label: offensive language
input: "@TheoMaxximus: Chrus still supahot tho" *walks up to teacher with test, drops test in that bitch fohead* http://t.co/nb5VgdssgI : label: hate speech
input: @aasthaxoxo oh shut your mouth i was talkin about the morning you fag : label: offensive language
input: Dem hoes be peeping out my dougie &#128131; : label: offensive language
input: @UberFacts studies show that you're a faggot : label: offensive language
input: I don't want sex bitch I only want ya neck bitch lol : label: offensive language
input: RT @_RealThickJazz: "@1inkkofrosess: Makem eat my ass since eating pussy ain't major nomore &#128175;&#128553;&#128514;&#9996;&#65039;&#128584;&#128129;&#128079;&#128522;" &#128514;&#128